In [2]:
!pip install secretsharing
!pip install argon2_cffi
!pip install passlib
!pip install pycrypto

  Running setup.py bdist_wheel for secretsharing ... - done
  Stored in directory: /root/.cache/pip/wheels/09/c6/4a/c97772071806a92c6e60a6aa5811dfb25c909ca29f37b03127
Successfully built secretsharing
    100% |████████████████████████████████| 102kB 6.0MB/s 
    100% |████████████████████████████████| 501kB 19.8MB/s 
    100% |████████████████████████████████| 450kB 12.4MB/s 
  Running setup.py bdist_wheel for pycrypto ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto


In [0]:
import os,sys,argparse,tempfile,shutil

# notes on secretsharing module below:
# - sudo -H pip install secret-sharing is needed first
# - secretsharing uses /dev/random by default, which is slow as it
#   gathers entropy from OS events - that's not only slow, but can
#   also frequently block, to get around this edit the source and
#   change it to use /dev/urandom which won't block
#   source to edit for me was:
#   /usr/local/lib/python2.7/dist-packages/secretsharing/entropy.py  
import secretsharing as sss

# for JSON output
#import jsonpickle # install via  "$ sudo pip install -U jsonpickle"

# for hashing passwords
from hashlib import sha256
import hashlib
# needed for these: sudo -H pip install passlib argon2_cffi
from passlib.hash import pbkdf2_sha256,argon2,sha512_crypt,sha1_crypt

# for non-security sensitive random numbers
from random import randrange

# for encrypting you need: sudo -H pip install pycrypto
import base64
from Crypto.Cipher import AES
from Crypto import Random
import json

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
#drive/My Drive/test/level1.json
#drive/My Drive/group_prac/code/00093_chapeaua.json
with open('drive/My Drive/group_prac/Layer_6/JSON/level6.json') as f:
    data = json.load(f)

raw_cypher=data["ciphertext"]
hashes=data["hashes"]
shares=data["shares"]
print(hashes)

[u'$pbkdf2-sha256$29000$rRWCUOrdm3Mu5Xxvbe19Dw$KSBQWjzKjTyGyRFhGCFFIYP0UFeJY9jFsP/XfQ.TnFk', u'$pbkdf2-sha256$29000$V.pdq5USAoBQirHWGuNcSw$KlGvdWV8dXp3yq6eSqTA8mofs9EpsusuubIHlOhT100', u'$argon2i$v=19$m=102400,t=2,p=8$J6SUkhJCyPk/B0BoLSXkXA$m2lXal4ENXO7BUv83vH9TA', u'$sha1$480000$PI3BOz9U$LP5ChUp9F6yEUQ.iBzJaH8/msk2n', u'$argon2i$v=19$m=102400,t=2,p=8$SYmx9l7LWcv53/v/37tXCg$e8UjXBvw2P+gL5HuXZNkag', u'$sha1$480000$6UwGoTnu$FR8S54grPglc40r6wt9vb7Blsl4D', u'$argon2i$v=19$m=102400,t=2,p=8$klJqjREiBGDsPccY47x3jg$oEMbHSr+8EUItdQt+GKRnA', u'$argon2i$v=19$m=102400,t=2,p=8$kNK6954z5tx7792b07pXig$zDjpW/LtBQw8EOQkrgpukA', u'$argon2i$v=19$m=102400,t=2,p=8$gpBSqvVei5GS8v5/L4VQCg$fw0pan3UJeeDmln/C65lxA', u'$pbkdf2-sha256$29000$0/ofw3jPWYsxZgxBaG1NyQ$opGAbwJ37zK9XWIflua2hiCYFFYDou/KFNinJOsQhsw', u'$argon2i$v=19$m=102400,t=2,p=8$uHfu/b+XUgoBwHivFcJYCw$H+YVm0+Ju+kPH74noPJKLQ', u'$pbkdf2-sha256$29000$M4ZwLoXQ.v9fS.n9H6P03g$tsnZYsLPzCE14Qn2HvM9s4W35T65LhDA/0D8Fro3.JE', u'$argon2i$v=19$m=102400,t=2,p=8$eY

In [0]:
#drive/My Drive/test/pass.potfile
with open("drive/My Drive/group_prac/Layer_6/potfile/L6.txt") as f:
    passwords = f.readlines()
passwords = [x.strip() for x in passwords]

In [8]:
inds=[]
passwords_list=[]
for ps in range(len(passwords)):
  inds.append(hashes.index(passwords[ps].split(":")[0]))
  passwords_list.append(passwords[ps].split(":")[1])
print(len(inds))
print(len(passwords_list))

70
70


In [9]:

shares_list=[]

for x in shares:
  shares_list.append(str(x))

print((passwords_list))
print((inds))
print(len(passwords_list))


['nuguit', 'miccai', 'euleen', '721215', '449193', '089580', 'zxc326', 'v2shot', 'sblady', 's14883', 's14s14', 'rotexs', 'unvme25', 'flautin', 'ben3504', 'KATIRIA', '8912866', 'yamequi', 'tellyl1', 'slomorn', 'sep0228', 'rit7878', '1aashley', 'prinner8', 'hippie25', '19971015', 'tiedye13', 'suzannev', 'sng12345', 'skate177', 'rubbaguy', 'rotarota', 'roping10', '12qw12as', '540622', '03ayit', '6434656', 'baby0213', 'j1a2s3d4', 'HONEEY', 'envisage', 'ryanjn1', 'liz5482', 'seren16', 'riptony1', 'nema431j', 'rosatqm', 'sazan15', 'karma82', 'sage34', 'sexyfun1', 'nanabev', 'someoya2', 'zars1991', 'tmgoeg94', 'Star2006', 'us121406', 'wcony21', 'sibatoh', 'subarusd', 'ssr1980', 'u4soh9q7', 'XggVUr', 'tfd157', 'tricina', 'tcjb05', 'trinelle', 'zaijo1', 'troya33', 'sobiti']
[154, 91, 83, 39, 69, 112, 142, 134, 127, 11, 158, 25, 160, 49, 88, 84, 155, 117, 1, 50, 137, 122, 51, 14, 64, 28, 56, 0, 128, 38, 40, 13, 97, 100, 76, 67, 78, 55, 133, 3, 79, 61, 110, 123, 89, 73, 70, 57, 48, 95, 136, 37, 1

In [10]:
import os,sys,argparse,tempfile,shutil 

# notes on secretsharing module below:
# - sudo -H pip install secret-sharing is needed first
# - secretsharing uses /dev/random by default, which is slow as it
#   gathers entropy from OS events - that's not only slow, but can
#   also frequently block, to get around this edit the source and
#   change it to use /dev/urandom which won't block
#   source to edit for me was:
#   /usr/local/lib/python2.7/dist-packages/secretsharing/entropy.py  
import secretsharing as sss

# for hashing passwords
from hashlib import sha256

# for encrypting you need: sudo -H pip install pycrypto
import base64
from Crypto.Cipher import AES
from Crypto import Random
BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)

# our cs7ns1-specific functions for shamir-like sharing
def pxor(pwd,share):
  '''
    XOR a hashed password into a Shamir-share
    1st few chars of share are index, then "-" then hexdigits
    we'll return the same index, then "-" then xor(hexdigits,sha256(pwd))
    we truncate the sha256(pwd) to if the hexdigits are shorter
    we left pad the sha256(pwd) with zeros if the hexdigits are longer
    we left pad the output with zeros to the full length we xor'd
  '''
  words=share.split("-")
  hexshare=words[1]
  slen=len(hexshare)
  hashpwd=sha256(pwd).hexdigest()
  hlen=len(hashpwd)
  outlen=0
  if slen<hlen:
      outlen=slen
      hashpwd=hashpwd[0:outlen]
  elif slen>hlen:
      outlen=slen
      hashpwd=hashpwd.zfill(outlen)
  else:
      outlen=hlen
  xorvalue=int(hexshare, 16) ^ int(hashpwd, 16) # convert to integers and xor 
  paddedresult='{:x}'.format(xorvalue)          # convert back to hex
  paddedresult=paddedresult.zfill(outlen)       # pad left
  result=words[0]+"-"+paddedresult              # put index back
  return result

def pwds_shares_to_secret(kpwds,kinds,diffs):
  '''
      take k passwords, indices of those, and the "public" shares and 
      recover shamir secret
  '''
  shares=[]
  for i in range(0,len(kpwds)):
    shares.append(pxor(kpwds[i],diffs[kinds[i]]))
  secret=sss.SecretSharer.recover_secret(shares)
  return secret

#pwds = ['hello', 'bye', 'hi there']
#diffs = ['1-af52d4a9c32', '2-af52d4a9c32','3-af52d4a9c32']
#inds = [0, 1, 2]
temp=[]
for i in range(1,len(passwords_list)):
  key=pwds_shares_to_secret(passwords_list[0:i], inds[0:i], shares_list)
  #print(i,"----",key)
  temp.append(key)
print((key))

d8cff6776750a21b51ec16b3112aa41a


In [0]:
unpad = lambda s: s[:-ord(s[len(s) - 1:])]
def decrypt1( key, enc ):
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv )
    return unpad(cipher.decrypt( enc[16:] ))

In [12]:
x=decrypt1(key.zfill(32).decode('hex'),raw_cypher)
#encode("utf-8")
print type(x)
print(x)
#print x.zfill(32).decode('hex')

#decrypt(raw_cypher,key.zfill(32).decode('hex'))

<type 'str'>
{
  "ciphertext": "...",
  "easteregg": "keyboard patterns",
  "hashes": [
    "$pbkdf2-sha256$29000$vvdey/lf6937/9/7v5dSKg$LYxP/Bx1mXuo8CapFp0ayKdZKBYsuthxZX1VaTqvCzU",
    "$sha1$480000$clOYrHve$rxzu8DVgtEn2HC6CgcNgmT6HfjLc",
    "$sha1$480000$..vzve1b$TUp9tuoOIK2sESr9ItgkcxQWjgiG",
    "$argon2i$v=19$m=102400,t=2,p=8$/j8HYExp7X1vDaGU8h7DuA$GyMFqo8x4ASRTeDw7iSvHg",
    "$sha1$480000$AQf.m9BG$OwbzDD9.QgAqSvU1NYCPstSXy1CT",
    "$pbkdf2-sha256$29000$bc05J.Sc834PIQTAGCNEyA$H5OETEzDIOJ.UfKUP1Y01DbTlx3DJ3GPUdL//6tutWI",
    "$6$rounds=656000$LOt7P.hpz.vY4V9d$pAO0XoL37zApi2kMIT9v8JjUXazPIGmFVtHZMhfzNyA5h2SuD544q4bi7MvH/1bI0go7Y.YHTAitaQsVhrPr./",
    "$sha1$480000$eecLApy7$kfDYaKMqsph1U4LoTYMs2xf3EKTT",
    "$argon2i$v=19$m=102400,t=2,p=8$5rzX2rtXqnVOqdWakxKi1A$HBTVOAl7T49tDcjtaFr1OQ",
    "$argon2i$v=19$m=102400,t=2,p=8$oTSGkJJSqjWmFAJAyDmH8A$TXlotQbXSEzMFfSchzXDNw",
    "$pbkdf2-sha256$29000$ei9lLKVUam2tNab0fm9tzQ$fKuXeuqSYXnTabFkQDvmAzx4WYa3PjWDoc9dPaoVccg",
    "$argon2i$v

In [0]:
with open('drive/My Drive/group_prac/Layer_2/JSON/level2.json', 'w') as outfile:
    for n in x:
        outfile.writelines(n)

In [0]:
with open('drive/My Drive/group_prac/Layer_2/JSON/level2.json') as f2:
    data2 = json.load(f2)

raw_cypher=data["ciphertext"]
new_hashes=data["hashes"]
new_shares=data["shares"]
print(hashes)

with open('new_hashes.txt', 'w') as f:
  for x in hashes:
    f.writelines("%s\n" % str(x))

files.download('new_hashes.txt')

[u'$pbkdf2-sha256$29000$VColpDTmXCslhNAaoxSilA$Yrvin35CwBxCNWSjCyKDGGAcKYKeuBTqvSAoRqRYy3k', u'$6$rounds=656000$H5lofkiCbBcqQ86A$j6ePTG0eEbsrMu1ojDPHnOBb4j58DHyCiD/xev8q4jTmscgjNOUZQMv1TRQSwP0x0k3qYmEwZLKDQgum6ZQLu0', u'$6$rounds=656000$IFS0EYDDJP3Pc5pb$u/fnzQ1xDiEHj1Hw62MMSFfbOBUEpYJLzq7h7X.Ffu4z.Gy0KA9qMhYYQZhcDGLwpM0l87rId72iVzs9zJeWy/', u'$argon2i$v=19$m=102400,t=2,p=8$mnMOQQgBgPA+5xyjtPaeEw$UhYqe2QcYe9giS8GVZJ5Sw', u'$pbkdf2-sha256$29000$zxkj5DwH4FwLISSkFEIIIQ$Q1.6N9R.jEO86M0f46CPRfbWATT5YRRtFT5JxFHj0yo', u'$sha1$480000$DWCWaD5z$EE5bpPpdVxlsgpgOnPWNj17dQi/c', u'$argon2i$v=19$m=102400,t=2,p=8$slYKAeC8V8oZwxgDACCEkA$7fyy1uiALKj8s7I/0Kv8zw', u'$6$rounds=656000$oq24avenUY3/elEZ$wyNmx1KIo5v7rMLXetiwyc.mwOvO3nzbYsvO4bz6eEt3WyjQK8nsUfOq1FLByhC7TWqyYarbzBMq2R4yTpq6X1', u'$6$rounds=656000$xvK3Y4tUT7Yew0yV$WIrzjskel46Jm6zgzSA4yFK5tuV9Z1u3kbbWhe7aDCh8j1bxJKC3fqwgMOMsE/4MLJ8UWQLpZfoaFiUhNC1ak/', u'$sha1$480000$YT8loO16$zzhYIPDJnVRylmn1.qGlAK7yG01A', u'$argon2i$v=19$m=102400,t=2,p=8$PqdUyvkfY8

In [0]:
from google.colab import files

with open('hashes.txt', 'w') as f:
  for x in hashes:
    f.writelines("%s\n" % str(x))

files.download('hashes.txt')